# World Inequality and Happiness Analysis

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

## Data Preparation

The `WID_countries.csv` file contains all the country codes used to label the country-specific `csv`s as well as some metadata on countries.

In [ ]:
countries = pd.read_csv(
    'source_data/WID/WID_countries.csv',
    sep=';',
    keep_default_na=False,
    na_values=['']
)

The variables we are interested in are as follows. For more information on each variable, see the [WID Codes Dictionary](https://wid.world/codes-dictionary/) and the [README](./source_data/WID/README.md).

- LCU per USD: This is the market exchange rate of the country's local currency units (LCU) to US dollars (USD). This is necessary to convert economic variables from the country's local currency into USD so that they can be compared between countries.
- Net national income per capita
- Average primary income: This refers to income that is earned through the production of goods and services. It includes the income generated by individuals or entities from factors of production such as labor, capital, and land.
  - Household & non-profits
  - Corporate
  - Government
- Average secondary income: This, also known as transfer income or redistributive income, consists of transfers of money or goods between individuals, households, or countries. These transfers are often aimed at redistributing income or providing financial assistance.
  - Household & non-profits
  - Corporate
  - Government
- Average household wealth
  - All households
  - Bottom 99% of households
  - Top 1% of households
- Social security benefits
- Share of net national income
  - Labor
  - Corporate

In [ ]:

variables = [
    # (Column Name, Variable Name, Percentile)
    ('lcu_per_usd', 'xlcusxi999', 'p0p100'),
    ('income_per_capita', 'anninci999', 'p0p100'),
    ('avg_primary_house_npish_income', 'aprihni999', 'p0p100'),
    ('avg_secondary_house_npish_income', 'asechni999', 'p0p100'),
    ('avg_primary_corp_income', 'apricoi999', 'p0p100'),
    ('avg_secondary_corp_income', 'aseccoi999', 'p0p100'),
    ('avg_primary_govt_income', 'aprigoi999', 'p0p100'),
    ('avg_secondary_govt_income', 'asecgoi999', 'p0p100'),
    ('avg_house_wealth', 'ahwealj992', 'p0p100'),
    ('avg_house_wealth_bot99', 'ahwealj992', 'p0p99'),
    ('avg_house_wealth_top1', 'ahwealj992', 'p99p100'),
    ('ss_benefits', 'assbhni999', 'p0p100'),
    ('labor_share_income', 'wlabshi999', 'p0p100'),
    ('capital_share_income', 'wcapshi999', 'p0p100')
]

Using the country codes listed in `WID_countries.csv` we're able to collect all the necessary information from the individual `WID_data_XX.csv` files, where `XX` is the country code. For more information on these files see the [README](./source_data/WID/README.md). In these files, each variable is a record, and there's no guarantee than any given country will have that variable defined. Moreover, different countries may have these variables defined for different years. We want to obtain the highest possible number of data points. To this end, we will have to inspect each country and see what is defined and for what years the variables are defined, and select a year that maximizes the number of data points.

When this is done, the best year available to us within the range $[2012, 2022]$ is 2015.

In [ ]:
# The following dictionary will have keys that are lists of countries which
# have all the variables we want for the given year key.
years = {year: [] for year in range(2012, 2023)}

# We want to restrict the country codes to full countries, excluding
# sub-regions. This can be done by only considering codes of length 2.
country_codes = [code for code in countries.alpha2 if len(code) == 2]

for country_code in country_codes:
    country_data = pd.read_csv(
        f'source_data/WID/WID_data_{country_code}.csv',
        sep=';',
        keep_default_na=False,
        na_values=['']
    )

    def get_variable_years(variable, percentile):
        """Obtain the years a given variable is defined for."""
        values = country_data.loc[
            (country_data.variable == variable)
            & (country_data.percentile == percentile)
        ]

        return set(values['year']) & years.keys()

    # This will contain the years for which all variables are defined.
    years_defined = {year for year in range(2012, 2023)}

    for _, variable, percentile in variables:
        years_defined = years_defined & get_variable_years(variable, percentile)
        if not years_defined:
            break

    for year in years_defined:
        years[year].append(country_code)

# Obtain the greatest year of those for which the most countries have all the
# relevant variables defined.
best_year = max([year for year, codes in years.items() if len(codes) == max([len(codes) for codes in years.values()])])

Now we can go through the countries of the best year and obtain the relevant data for that country, converting variable records into `DataFrame` columns.

In [ ]:
# This will contain all the variables for each country.
data = []

# We will be using data from the World Happiness Report from 2015.
happiness_data = pd.read_excel(
    'source_data/WHR2015/Chapter2OnlineData_Expanded-with-Trust-and-Governance.xlsx',
    sheet_name='Data for Figure2.2',
    skiprows=3
)

for country_code in years[best_year]:
    country_data = pd.read_csv(
        f'source_data/WID/WID_data_{country_code}.csv',
        sep=';',
        keep_default_na=False,
        na_values=['']
    )

    country_name = countries.loc[countries.alpha2 == country_code].shortname.iloc[0]
    
    happiness_scores = happiness_data.loc[happiness_data.country == country_name]['Ladder score']
    happiness_score = None
    if happiness_scores.empty:
        continue
    else:
        assert(len(happiness_scores) == 1)
        happiness_score = happiness_scores.iloc[0]

    def read_variable(variable, percentile):
        """Read the value of a variable."""
        values = country_data.loc[
            (country_data.variable == variable)
            & (country_data.percentile == percentile)
            & (country_data.year == best_year)
        ]

        if values.empty:
            return None

        assert(len(values) == 1)
        return values.iloc[0].value

    skip_row = False
    row = [country_code, happiness_score]
    for _, variable, percentile in variables:
        x = read_variable(variable, percentile)
        
        # We want to skip any countries that don't have all variables defined.
        if not x:
            skip_row = True
            break
        
        row.append(x)

    if skip_row:
        continue

    data.append(row)

data = pd.DataFrame.from_records(
    data,
    columns=['country', 'happiness_score'] + [col for col, _, _ in variables]
)

We'll be working with these countries:

In [ ]:
data.country

All the variables for each country are given in LCU (local currency units). We have to use the exchange rate into USD to normalize all the countries' variables into USD.

In [ ]:
def norm_col_currency(df, col):
    df[col] = df[col] / df.lcu_per_usd

norm_col_currency(data, 'income_per_capita')
norm_col_currency(data, 'avg_primary_house_npish_income')
norm_col_currency(data, 'avg_secondary_house_npish_income')
norm_col_currency(data, 'avg_primary_corp_income')
norm_col_currency(data, 'avg_secondary_corp_income')
norm_col_currency(data, 'avg_primary_govt_income')
norm_col_currency(data, 'avg_secondary_govt_income')
norm_col_currency(data, 'avg_house_wealth')
norm_col_currency(data, 'avg_house_wealth_bot99')
norm_col_currency(data, 'avg_house_wealth_top1')
norm_col_currency(data, 'ss_benefits')

## Data Analysis

The following function will allow us to easily plot trendlines.

In [ ]:
def plot_trendline(ax, x, y, **kwargs):
    """Plot a linear trendline."""
    m, b = np.polyfit(x, y, deg=1)
    ax.plot(x, x * m + b, **kwargs)

plt.Axes.trendline = plot_trendline

### Income

Unsurprisingly, net national income is correlated with average primary and secondary incomes of the household & NPISH, corporate, and government sectors.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12.8, 4.8), sharex=True, sharey=True)

fig.suptitle('Primary and Secondary Incomes by Net National Income per Capita')
fig.supxlabel('Net National Income per Capita')

ax1.set_title('Primary Income')
ax1.set_ylabel('Avg. Primary Income')
ax1.xaxis.set_major_formatter('${x:1.0f}')
ax1.yaxis.set_major_formatter('${x:1.0f}')
ax1.scatter(data.income_per_capita, data.avg_primary_house_npish_income)
ax1.scatter(data.income_per_capita, data.avg_primary_corp_income)
ax1.scatter(data.income_per_capita, data.avg_primary_govt_income)
ax1.trendline(data.income_per_capita, data.avg_primary_house_npish_income)
ax1.trendline(data.income_per_capita, data.avg_primary_corp_income)
ax1.trendline(data.income_per_capita, data.avg_primary_govt_income)

ax2.set_title('Secondary Income')
ax2.set_ylabel('Avg. Secondary Income')
ax2.xaxis.set_major_formatter('${x:1.0f}')
ax2.yaxis.set_major_formatter('${x:1.0f}')
ax2.scatter(data.income_per_capita, data.avg_secondary_house_npish_income)
ax2.scatter(data.income_per_capita, data.avg_secondary_corp_income)
ax2.scatter(data.income_per_capita, data.avg_secondary_govt_income)
ax2.trendline(data.income_per_capita, data.avg_secondary_house_npish_income)
ax2.trendline(data.income_per_capita, data.avg_secondary_corp_income)
ax2.trendline(data.income_per_capita, data.avg_secondary_govt_income)

fig.legend(['Household & NPISH', 'Corporate', 'Government'])

There doesn't appear to be any correlation between national income and labor or capital shares of national income.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Labor and Capital Shares of Net National Income')

ax.scatter(data.income_per_capita, data.labor_share_income)
ax.scatter(data.income_per_capita, data.capital_share_income)
ax.set_xlabel('Net National Income per Capita')
ax.set_ylabel('Share of National Income')
ax.legend(['Labor', 'Capital'])
ax.xaxis.set_major_formatter('${x:1.0f}')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))

stats.pearsonr(data.income_per_capita, data.labor_share_income), stats.pearsonr(data.income_per_capita, data.capital_share_income)

There is a positive correlation between national income and social security benefits.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Social Security Benefits by Net National Income')

ax.scatter(data.income_per_capita, data.ss_benefits)
ax.trendline(data.income_per_capita, data.ss_benefits)
ax.set_xlabel('Net National Income per Capita')
ax.xaxis.set_major_formatter('${x:1.0f}')
ax.set_ylabel('Social Security Benefits')
ax.yaxis.set_major_formatter('${x:1.0f}')

stats.pearsonr(data.income_per_capita, data.ss_benefits)

### Income and Wealth

There is a correlation between average household wealth and national income.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Avg. Household Wealth by Net National Income')

ax.scatter(data.income_per_capita, data.avg_house_wealth)
ax.trendline(data.income_per_capita, data.avg_house_wealth)
ax.set_xlabel('Net National Income per Capita')
ax.set_ylabel('Avg. Household Wealth')
ax.xaxis.set_major_formatter('${x:1.0f}')
ax.yaxis.set_major_formatter('${x:1.0f}')

stats.pearsonr(data.income_per_capita, data.avg_house_wealth)

Interestingly, this correlation is stronger for the top 1% of household wealth.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Avg. Household Wealth by Net National Income')

ax.scatter(data.income_per_capita, data.avg_house_wealth_bot99)
ax.scatter(data.income_per_capita, data.avg_house_wealth_top1)
ax.trendline(data.income_per_capita, data.avg_house_wealth_bot99)
ax.trendline(data.income_per_capita, data.avg_house_wealth_top1)
ax.legend(['Bottom 99%', 'Top 1%'])
ax.set_xlabel('Net National Wealth per Capita')
ax.set_ylabel('Avg. Household Wealth')
ax.xaxis.set_major_formatter('${x:1.0f}')
ax.yaxis.set_major_formatter('${x:1.0f}')

stats.pearsonr(data.income_per_capita, data.avg_house_wealth_bot99), stats.pearsonr(data.income_per_capita, data.avg_house_wealth_top1)

The correlation is still present, but much weaker, for the bottom 99% of households. Notice the scale of the y-axis has changed tremendously.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Avg. Household Wealth of Bottom 99% by Net National Income')

ax.scatter(data.income_per_capita, data.avg_house_wealth_bot99)
ax.trendline(data.income_per_capita, data.avg_house_wealth_bot99)
ax.set_xlabel('Net National Income per Capita')
ax.set_ylabel('Avg. Household Wealth')
ax.xaxis.set_major_formatter('${x:1.0f}')
ax.yaxis.set_major_formatter('${x:1.0f}')

### Happiness

Hapoiness scores are correlated positively with both primary and secondary income in all sectors. The weakest correlation is in the corporate sector.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12.8, 4.8), sharex=True, sharey=True)

fig.suptitle('Happiness by Income')
fig.supylabel('Happiness Score')

ax1.set_title('Happiness by Primary Income')
ax1.set_xlabel('Avg. Primary Income')
ax1.xaxis.set_major_formatter('${x:1.0f}')
ax1.scatter(data.avg_primary_house_npish_income, data.happiness_score)
ax1.scatter(data.avg_primary_corp_income, data.happiness_score)
ax1.scatter(data.avg_primary_govt_income, data.happiness_score)
ax1.trendline(data.avg_primary_house_npish_income, data.happiness_score)
ax1.trendline(data.avg_primary_corp_income, data.happiness_score)
ax1.trendline(data.avg_primary_govt_income, data.happiness_score)

ax2.set_title('Happiness by Secondary Income')
ax2.set_xlabel('Avg. Secondary Income')
ax2.xaxis.set_major_formatter('${x:1.0f}')
ax2.scatter(data.avg_secondary_house_npish_income, data.happiness_score)
ax2.scatter(data.avg_secondary_corp_income, data.happiness_score)
ax2.scatter(data.avg_secondary_govt_income, data.happiness_score)
ax2.trendline(data.avg_secondary_house_npish_income, data.happiness_score)
ax2.trendline(data.avg_secondary_corp_income, data.happiness_score)
ax2.trendline(data.avg_secondary_govt_income, data.happiness_score)

fig.legend(['Household & NPISH', 'Corporate', 'Government'])

(stats.pearsonr(data.avg_primary_house_npish_income, data.happiness_score),
 stats.pearsonr(data.avg_primary_corp_income, data.happiness_score),
 stats.pearsonr(data.avg_primary_govt_income, data.happiness_score),
 stats.pearsonr(data.avg_secondary_house_npish_income, data.happiness_score),
 stats.pearsonr(data.avg_secondary_corp_income, data.happiness_score),
 stats.pearsonr(data.avg_secondary_govt_income, data.happiness_score))

Happiness is negatively correlated with capital share of national income and postively correlated with labor share of national income, although the correlation is not incredibly strong.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Happiness by Share of National Income')

ax.scatter(data.labor_share_income, data.happiness_score)
ax.scatter(data.capital_share_income, data.happiness_score)
ax.trendline(data.labor_share_income, data.happiness_score)
ax.trendline(data.capital_share_income, data.happiness_score)
ax.set_xlabel('Share of National Income')
ax.set_ylabel('Happiness Score')
ax.legend(['Labor', 'Capital'])
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1))

stats.pearsonr(data.labor_share_income, data.happiness_score), stats.pearsonr(data.capital_share_income, data.happiness_score)

Happiness is positively correlated with social security benefits.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Happiness by Social Security Benefits')

ax.scatter(data.ss_benefits, data.happiness_score)
ax.trendline(data.ss_benefits, data.happiness_score)
ax.set_xlabel('Social Security Benefits')
ax.xaxis.set_major_formatter('${x:1.0f}')
ax.set_ylabel('Happiness Score')

stats.pearsonr(data.ss_benefits, data.happiness_score)

Happiness is positively correlated with net national income per capita.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Happiness by Net National Income')

ax.scatter(data.income_per_capita, data.happiness_score)
ax.trendline(data.income_per_capita, data.happiness_score)
ax.set_xlabel('Net National Income per Capita')
ax.set_ylabel('Happiness Score')
ax.xaxis.set_major_formatter('${x:1.0f}')

stats.pearsonr(data.income_per_capita, data.happiness_score)

Happiness is positvely correlated with average household wealth.

In [ ]:
fig, ax = plt.subplots()

fig.suptitle('Happiness by Avg. Household Wealth')

ax.scatter(data.avg_house_wealth, data.happiness_score)
ax.trendline(data.avg_house_wealth, data.happiness_score)
ax.set_xlabel('Avg. Household Wealth')
ax.set_ylabel('Happiness Score')
ax.xaxis.set_major_formatter('${x:1.0f}')

stats.pearsonr(data.avg_house_wealth, data.happiness_score)

Breaking this down into bottom 99% and top 1%, we can see that happiness is very strongly correlated with the household wealth of the bottom 99% of households. It is also correlated with the average household wealth of the top 1%, but not nearly as strongly.

In [ ]:
fig, ax = plt.subplots(figsize=(12.8, 4.8))

fig.suptitle('Happiness by Avg. Household Wealth')

ax.scatter(data.avg_house_wealth_bot99, data.happiness_score)
ax.scatter(data.avg_house_wealth_top1, data.happiness_score)
ax.trendline(data.avg_house_wealth_bot99, data.happiness_score)
ax.trendline(data.avg_house_wealth_top1, data.happiness_score)
ax.legend(['Bottom 99%', 'Top 1%'])
ax.set_xlabel('Avg. Household Wealth')
ax.set_ylabel('Happiness Score')
ax.xaxis.set_major_formatter('${x:1.0f}')

(stats.pearsonr(data.avg_house_wealth_bot99, data.happiness_score),
 stats.pearsonr(data.avg_house_wealth_top1, data.happiness_score))